In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.0 MB/s eta 0:00:00


In [ ]:
import torchvision

In [ ]:
import cv2

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt


import pydicom as dicom

from sklearn.preprocessing import LabelEncoder

from torch.utils.data import Dataset, DataLoader, Subset

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

from torch.autograd import Variable
from sklearn.model_selection import StratifiedKFold,GroupKFold

In [ ]:
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else:
  DEVICE = torch.device('cpu')

In [ ]:
BATCH_SIZE = 100
EPOCHS = 10
FOLDS = 5

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/파이널 프로젝트/data1/train.csv")
base_path = "/content/drive/MyDrive/파이널 프로젝트/data1/train_processed/"
all_paths = []
for k in range(len(train)):
  row = train.iloc[k,:]
  all_paths.append(base_path + str(row.patient_id)+ "/" + str(row.image_id) + ".png")

train['path'] = all_paths

In [ ]:
train.path.isnull().sum()

0

In [ ]:
class Dataset(Dataset):
  def __init__(self, dataframe, istrain):
    self.dataframe = dataframe
    self.istrain = istrain

    if self.istrain == True:
      self.transform = T.Compose([T.RandomResizedCrop((224, 224), scale = (0.2, 1.0))])
    else:
      self.transform = T.Compose([T.ToTensor()])

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, index):
    image_path = self.dataframe['path'][index]
    image = cv2.imread(image_path)
    image = image.view(-1, image.shape[0], image.shape[1])

    image = self.transform(image)
    if self.istrain == True:
      return image, self.dataframe['cancer'][index]
    else:
      return image



NameError: ignored

In [ ]:
dataset = Dataset(train, True)
dataloader = DataLoader(dataset, batch_size= BATCH_SIZE, shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN , self).__init__()
        self.conv1 = nn.Conv2d(1 , 32 , kernel_size = 5)
        self.conv2 = nn.Conv2d(32 , 32 , kernel_size = 5)
        self.conv3 = nn.Conv2d(32 , 64 , kernel_size = 5)
        self.fc1 = nn.Linear(2 * 2 * 43264 , 512)
        self.fc2 = nn.Linear(512 , 2)

    def forward(self , x):
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x) , 2))
        x = F.dropout(x , p = 0.5 , training = self.training)
        x = F.relu(F.max_pool2d(self.conv3(x) , 2))
        x = F.dropout(x , p = 0.5 , training = self.training)
        x = x.view(-1 , 2 * 2 * 43264)
        x = F.relu(self.fc1(x))
        x = F.dropout(x , training = self.training)
        x = self.fc2(x)
        return F.log_softmax(x , dim = 1)

In [ ]:
model = CNN().to(DEVICE)

def fit():
    
    group_fold = GroupKFold(n_splits = FOLDS)
    
    k_folds = group_fold.split(X = np.zeros(len(train)), 
                               y = train['cancer'], 
                               groups = train['patient_id'].tolist())
    
    for i, (train_index, valid_index) in enumerate(k_folds):
        print(len(train_index) , len(valid_index))
        
        
        train_data = train.iloc[train_index].reset_index(drop=True)
        valid_data = train.iloc[valid_index].reset_index(drop=True)

        # Create Data instances
        train_dataset = Dataset(train_data , True)
        valid_dataset = Dataset(valid_data , False)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

        optimizer = torch.optim.Adam(model.parameters())
        error = nn.CrossEntropyLoss()
        model.train()

        for epoch in range(EPOCHS):
            correct = 0
            for batch_idx , (X_batch , y_batch) in enumerate(train_loader):
                vX_batch = Variable(X_batch).float()
                vy_batch = Variable(y_batch)
                
                vX_batch = vX_batch.to(DEVICE)
                vy_batch = vy_batch.to(DEVICE)
                
                optimizer.zero_grad()
                output = model(vX_batch)
                loss = error(output , vy_batch)
                loss.backward()
                optimizer.step()

                predicted = torch.max(output.data , 1)[1]

                correct += (predicted == vy_batch).sum()

            print("Epoch: " , epoch , "\tLoss: " , loss.data , "\tAccuracy: " , round(float(correct*100) / float(len(train_index))) , 2)
            
        train_acc = round(float(correct*100) / float(len(train_index)) , 2)
        
        print("Train Accuracy: " , train_acc , "%" )
        
        model.eval()

        with torch.no_grad():
            for batch_idx , (X_batch , y_batch)  in enumerate(valid_loader):
                
                vX_batch = Variable(X_batch).float()
                vy_batch = Variable(y_batch)
                
                vX_batch = vX_batch.to(DEVICE)
                vy_batch = vy_batch.to(DEVICE)

                output = model(vX_batch)
                predicted = torch.max(output.data , 1)[1]
                correct += (predicted == vy_batch).sum()

            # Calculate accuracy
            valid_acc = round(float(correct*100) / float(len(valid_index)) , 2)
            
            print("Valid Accuracy: " , valid_acc , "%" )                 

In [ ]:
fit()

43763 10943
